In [1]:
import pandas as pd
import json
import config
from data_loader import load_hourly_solar_data
from data_loader import load_hourly_load_data
from simulation import run_simulation
from components.battery import Battery
from data_loader import load_test


In [124]:
with open("data/hourly_data.json", "r") as f:
    raw_data = json.load(f)

In [5]:
for field, content in raw_data.items():
    print(f"Field: {field}")
    print(f"Keys: {content.keys()}")
    break  # just check the first one

Field: roof_southwest
Keys: dict_keys(['timestamp', 'power_kwh'])


In [131]:
pd.to_datetime(pd.DataFrame(raw_data['roof_southwest'])['timestamp'][0])

Timestamp('2010-01-01 00:10:00')

In [19]:
selected_fields=config.SELECTED_FIELD_NAMES

hourly_data_dic = {
        field: pd.DataFrame({
            "power_kwh": data["power_kwh"]
        }, index=pd.to_datetime(data['timestamp']).floor('h'))
        for field, data in raw_data.items()
        if selected_fields is None or field in selected_fields
    }

In [7]:
selected_fields

['roof_southwest', 'backwall_t90']

In [18]:
hourly_data_dic = load_hourly_solar_data(
    filename="data/hourly_data.json",
    selected_fields=config.SELECTED_FIELD_NAMES
)

In [19]:
solar_df = sum(hourly_data_dic.values()).round(2)
solar_df.columns = ["power_kwh"]

In [ ]:
solar_df

,power_kwh
2010-01-01 00:00:00,0.0
2010-01-01 01:00:00,0.0
2010-01-01 02:00:00,0.0
2010-01-01 03:00:00,0.0
2010-01-01 04:00:00,0.0
...,...
2023-12-31 19:00:00,0.0
2023-12-31 20:00:00,0.0
2023-12-31 21:00:00,0.0
2023-12-31 22:00:00,0.0


: 

In [13]:
load_df = load_hourly_load_data("data/hourly_load_data_per_month.csv")

# Create battery
battery = Battery()

In [16]:
load_df

,power_kwh
2010-01-01 01:00:00,1
2010-01-01 02:00:00,1
2010-01-01 03:00:00,1
2010-01-01 04:00:00,1
2010-01-01 05:00:00,1
...,...
2023-12-31 19:00:00,1
2023-12-31 20:00:00,1
2023-12-31 21:00:00,1
2023-12-31 22:00:00,1


In [14]:
# Align dataframes
start = max(solar_df.index.min(), load_df.index.min())
end = min(solar_df.index.max(), load_df.index.max())
solar_df = solar_df[start:end]
load_df = load_df[start:end]

In [15]:
# Run simulation
results = run_simulation(solar_df, load_df, battery)

KeyError: "None of ['timestamp'] are in the columns"

In [39]:
raw_data = pd.read_csv("data/hourly_load_data_per_month.csv", sep=";", index_col='Month')

In [40]:
raw_data

,00:00,01:00,02:00,03:00,04:00,05:00,06:00,07:00,08:00,09:00,...,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00
Month,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,1,2,4,...,3,3,4,4,2,1,1,1,1,1
2,1,1,1,1,1,1,1,1,2,4,...,3,3,4,4,2,1,1,1,1,1
3,1,1,1,1,1,1,1,1,2,4,...,3,3,4,4,2,1,1,1,1,1
4,1,1,1,1,1,1,1,1,2,4,...,3,3,4,4,2,1,1,1,1,1
5,0,0,0,0,0,0,0,1,2,4,...,2,2,2,2,3,2,2,0,0,0
6,0,0,0,0,0,0,0,1,2,4,...,2,2,2,2,3,2,2,0,0,0
7,0,0,0,0,0,0,0,1,2,4,...,2,2,2,2,3,2,2,0,0,0
8,0,0,0,0,0,0,0,1,2,4,...,2,2,2,2,3,2,2,0,0,0
9,0,0,0,0,0,0,0,1,2,4,...,2,2,2,2,3,2,2,0,0,0


In [32]:
selected_fields = ['roof_southwest', 'roof_northeast']

with open("data/hourly_data.json", "r") as f:
    raw_data = json.load(f)

hourly_data_dic = {
    field: pd.DataFrame({
        "power_kwh": data["power_kwh"]
    }, index=pd.DataFrame(data).index)
    for field, data in raw_data.items()
    if selected_fields is None or field in selected_fields
}

In [31]:
pd.DataFrame(raw_data['roof_northeast']).index

RangeIndex(start=0, stop=122712, step=1)

In [94]:
import os
print(os.getcwd())

c:\Users\wmd327\Dropbox\Programming\solar_simulation
